In [109]:
# import packages
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import time
import torch
import torch.nn as nn
import torch.utils.data as Data
import torch.nn.functional as F
from sklearn.metrics import roc_auc_score,accuracy_score,classification_report

import sys

# Add the directory containing the module to the Python path
sys.path.append("C:\\Users\\sherl\\Downloads\\nit pendrive\\Deep-Learning-for-BCI-master\\Deep-Learning-for-BCI-master\\tutorial")

# Now you can import the module
import myimporter


import myimporter
from BCI_functions import *  # BCI_functions.ipynb contains some functions we might use multiple times in this tutorial
import warnings
warnings.filterwarnings('ignore')

In [110]:
import mne


In [111]:
%matplotlib inline
import os
import os.path as op
import mne
import numpy as np
import pandas as pd

In [112]:
def make_windowing(X,Y):
    X_new = []
    Y_new = []
    npt = 128 # .5 sec
    stride = 8 # around .1 sec
    ctr = 0
    for i in range(0,X.shape[0]):
        y = Y[i]
        a= X[i,:,:]
        a = a.transpose()
        a.shape
        val = 0
        kd=len(a)
        while val<=(len(a)-npt):
            x = a[val:val+npt,:]

            X_new.append(x.T)
            Y_new.append(y)
            val = val+stride
            
    return np.array(X_new),np.array(Y_new)

In [113]:
# Load .edf file
filename = "C:\\Users\\sherl\\Downloads\\Vipin_Apple.edf"
raw = mne.io.read_raw_edf(filename)


Extracting EDF parameters from C:\Users\sherl\Downloads\Vipin_Apple.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...


In [114]:
raw.resample(250, npad="auto")    # set sampling frequency to 256 points per second

<RawEDF | Vipin_Apple.edf, 39 x 3250 (13.0 s), ~1.0 MB, data loaded>

In [115]:
raw.filter(1, 45, fir_design='firwin', picks=['eeg'])  # band-pass filter from 1 to 30 frequency over just
                                                       # EEG channel and not EEG channel

Filtering raw data in 1 contiguous segment
Setting up band-pass filter from 1 - 45 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 1.00
- Lower transition bandwidth: 1.00 Hz (-6 dB cutoff frequency: 0.50 Hz)
- Upper passband edge: 45.00 Hz
- Upper transition bandwidth: 11.25 Hz (-6 dB cutoff frequency: 50.62 Hz)
- Filter length: 825 samples (3.300 s)



[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   4 out of   4 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done  39 out of  39 | elapsed:    0.0s finished


<RawEDF | Vipin_Apple.edf, 39 x 3250 (13.0 s), ~1.0 MB, data loaded>

In [116]:
raw.set_eeg_reference('average', projection=True).apply_proj()  # re-referencing with the virtual average reference

EEG channel type selected for re-referencing
Adding average EEG reference projection.
1 projection items deactivated
Average reference projection was added, but has not been applied yet. Use the apply_proj method to apply it.
Created an SSP operator (subspace dimension = 1)
1 projection items activated
SSP projectors applied...


<RawEDF | Vipin_Apple.edf, 39 x 3250 (13.0 s), ~1.0 MB, data loaded>

In [117]:
print(raw.info)
'''
Sampling frequency (sfreq)
Channel names (ch_names)
Channel types (ch_types)
Channel units (units)
High-pass and low-pass filter settings (highpass, lowpass)
EEG reference (ref)
EEG electrode locations (chs[idx]['loc'])
Measurement date and time (meas_date)
Projector information (projs)
Bad channels (bads)
Sensor positions (dig)
Trigger information (n_savesys and related fields)
'''

<Info | 8 non-empty values
 bads: []
 ch_names: COUNTER, INTERPOLATED, AF3, F7, F3, FC5, T7, P7, O1, O2, P8, T8, ...
 chs: 39 EEG
 custom_ref_applied: False
 highpass: 1.0 Hz
 lowpass: 45.0 Hz
 meas_date: 2017-02-01 12:10:40 UTC
 nchan: 39
 projs: Average EEG reference: on
 sfreq: 250.0 Hz
>


"\nSampling frequency (sfreq)\nChannel names (ch_names)\nChannel types (ch_types)\nChannel units (units)\nHigh-pass and low-pass filter settings (highpass, lowpass)\nEEG reference (ref)\nEEG electrode locations (chs[idx]['loc'])\nMeasurement date and time (meas_date)\nProjector information (projs)\nBad channels (bads)\nSensor positions (dig)\nTrigger information (n_savesys and related fields)\n"

In [118]:
epochs = mne.read_epochs("C:\\Users\\sherl\\Downloads\\1Eldo-epo.fif")
# epochs1 = mne.read_epochs('../preprocessed/1Eldo-epo.fif')
# epochs2 = mne.read_epochs('../preprocessed/2Eldo-epo.fif')

Reading C:\Users\sherl\Downloads\1Eldo-epo.fif ...
    Found the data of interest:
        t =    -203.12 ...    2000.00 ms
        0 CTF compensation matrices available
Not setting metadata
444 matching events found
No baseline correction applied
0 projection items activated


In [119]:
# data1 = epochs1.get_data()
# data2 = epochs2.get_data()
# data = np.array(data1.tolist()+data2.tolist())
data=epochs.get_data()

In [120]:
ica = mne.preprocessing.ICA(n_components=14, random_state=97, method='fastica')
ica.fit(epochs)                                      # Data decomposition with 50 components and fastica method.

Fitting ICA to data using 14 channels (please be patient, this may take a while)
Selecting by number: 14 components
Fitting ICA took 1.0s.


Method,fastica
Fit,21 iterations on epochs (125652 samples)
ICA components,14
Available PCA components,14
Channel types,eeg
ICA components marked for exclusion,—


In [121]:
ica.exclude = [11, 26, 29, 30, 33, 34, 35, 36, 38, 44, 48, 49, 0, 6, 17] 
                                        # Put all comonent which you want to remove containg inspected (manual) 
                                        # [11, 26, 29, 30, 33, 34, 35, 36, 38, 44, 48, 49], EOG [0] and ECG [6,17] components
                                        # Selected components are not real
ica.apply(epochs)                       # Channels can be reconstructed using the ICA object’s apply()

Applying ICA to Epochs instance
    Transforming to ICA space (14 components)
    Zeroing out 3 ICA components
    Projecting back using 14 PCA components


Number of events,444
Events,label_IS1-COLD: 15label_IS1-DOCTOR: 15label_IS1-EATING: 15label_IS1-LIGHT: 15label_IS1-PAIN: 15label_IS1-SICK: 15label_IS1-TOILET: 15label_IS1-TV: 15label_IS1-WATER: 13label_IS1-YES: 14label_IS2-COLD: 14label_IS2-DOCTOR: 15label_IS2-EATING: 15label_IS2-LIGHT: 15label_IS2-PAIN: 15label_IS2-SICK: 15label_IS2-TOILET: 14label_IS2-TV: 15label_IS2-WATER: 15label_IS2-YES: 15label_IS3-COLD: 14label_IS3-DOCTOR: 15label_IS3-EATING: 15label_IS3-LIGHT: 15label_IS3-PAIN: 15label_IS3-SICK: 15label_IS3-TOILET: 15label_IS3-TV: 15label_IS3-WATER: 15label_IS3-YES: 15
Time range,-0.203 – 2.000 s
Baseline,-0.203 – 0.000 s


In [122]:
epochs.save("C:\\Users\\sherl\\Downloads\\sub-006_prerprocessed.fif",overwrite=True)


Overwriting existing file.


In [123]:
import matplotlib
import os
import os.path as op
import mne
import numpy as np
import pandas as pd
# from mne_icalabel import label_components
# from mayavi import mlab
# %gui qt
# %matplotlib qt
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import GroupKFold,GridSearchCV,cross_val_score,cross_validate 
# Load necessary libraries
import mne
from mne.decoding import Vectorizer

from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline
from sklearn.model_selection import cross_val_score, train_test_split, GridSearchCV, StratifiedKFold
from sklearn.metrics import classification_report, accuracy_score, precision_recall_fscore_support

# Models
from sklearn import svm
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier


import sklearn.model_selection as model_selection
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score

In [124]:
imagined_words_dict ={ "SICK":0,
"COLD":1,
"PAIN":2,
"TOILET":3,
"EATING":4,
"WATER":5,
"LIGHT":6,
"DOCTOR":7,
"YES":8,
"TV":9
}

In [125]:
def set_label(epochss):
    labels = epochss.events[:, -1].copy()
    event_id = epochss.event_id
    for i in range(len(labels)):
        label_key =list(event_id.keys())[list(event_id.values()).index(labels[i])]
        start_index =label_key.find("-")
        label_key=label_key[start_index+1:]
        labels[i] =imagined_words_dict[label_key]
    return labels


In [126]:
# labels1 = set_label(epochs1)
# labels2 = set_label(epochs2)
# labels = np.array(labels1.tolist() + labels2.tolist())
labels = set_label(epochs)

In [127]:
# Using Skicit-learn to split data into training and testing sets
from sklearn.model_selection import train_test_split
def read_split_epoch(split):
    epochs = mne.read_epochs("C:\\Users\\sherl\\Downloads\\1Eldo-epo.fif")
    data = epochs.get_data()
    labels = set_label(epochs)
    # Split the data into training and testing sets
    train_epoch, test_epoch, tr_labels, ts_labels = train_test_split(data, labels,shuffle=True)
    return train_epoch, test_epoch, tr_labels, ts_labels

In [128]:
# print(no_feature*segment_length)

In [129]:
tr_ep_data,ts_ep_data,tr_labels,ts_labels = read_split_epoch(25)

Reading C:\Users\sherl\Downloads\1Eldo-epo.fif ...
    Found the data of interest:
        t =    -203.12 ...    2000.00 ms
        0 CTF compensation matrices available
Not setting metadata
444 matching events found
No baseline correction applied
0 projection items activated


In [130]:
tr_window_data,tr_window_labels = make_windowing(tr_ep_data,tr_labels)
ts_window_data,ts_window_labels = make_windowing(ts_ep_data,ts_labels)
#window_data,window_labels =data,labels



In [131]:
# train_feature, test_feature, train_label, test_label = train_test_split(data_seg_feature, data_seg_label, shuffle=True)

# train_epoch, test_epoch, tr_labels, ts_labels = train_test_split(data, labels, test_size = 0.25, random_state = 42)

# tr_ep_data,ts_ep_data,tr_labels,ts_labels


In [132]:
train_feature, test_feature, train_label, test_label =tr_ep_data,ts_ep_data,tr_labels,ts_labels

In [133]:
segment_length=283 # segment_length= sampling_freuncy* stride_length | 25=250*0.1
no_feature=14

In [134]:
# check if a GPU is available
with_gpu = torch.cuda.is_available()
if with_gpu:
    device = torch.device("cuda")
else:
    device = torch.device("cpu")
print('We are using %s now.' %device)

We are using cpu now.


In [135]:
train_fea_norm1 , test_fea_norm1 = train_feature, test_feature

In [136]:
BATCH_size = test_fea_norm1.shape[0] # use test_data as batch size

In [137]:
# feed data into dataloader
train_fea_norm1 = torch.tensor(train_fea_norm1).to(device)
train_label = torch.tensor(train_label.flatten()).to(device)
train_data = Data.TensorDataset(train_fea_norm1, train_label)
train_loader = Data.DataLoader(dataset=train_data, batch_size=BATCH_size, shuffle=False)

test_fea_norm1 = torch.tensor(test_fea_norm1).to(device)
test_label = torch.tensor(test_label.flatten()).to(device)

In [138]:
n_class = 10
no_feature = 14  # the number of the features
segment_length = 283  # selected time window; 
LR = 0.005  # learning rate
EPOCH = 8880 #after windowing
n_hidden = 128  # number of neurons in hidden layer
l2 = 0.001  # the coefficient of l2-norm regularization


In [139]:
# classifier
class LSTM(nn.Module):
    def __init__(self):
        super(LSTM, self).__init__()

        self.lstm_layer = nn.LSTM(
            input_size=no_feature,
            hidden_size=n_hidden,         # LSTM hidden unit
            num_layers=2,           # number of LSTM layer
            bias=True,
            batch_first=True,       # input & output will has batch size as 1s dimension. e.g. (batch, segment_length, no_feature)
        )

        self.out = nn.Linear(n_hidden, n_class)

    def forward(self, x):
        r_out, (h_n, h_c) = self.lstm_layer(x.float(), None)
        r_out = F.dropout(r_out, 0.3)

        test_output = self.out(r_out[:, -1, :]) # choose r_out at the last time step
        return test_output

lstm = LSTM()
lstm.to(device)
print(lstm)

optimizer = torch.optim.Adam(lstm.parameters(), lr=LR, weight_decay=l2)   # optimize all parameters
loss_func = nn.CrossEntropyLoss()

LSTM(
  (lstm_layer): LSTM(14, 128, num_layers=2, batch_first=True)
  (out): Linear(in_features=128, out_features=10, bias=True)
)


In [144]:
best_acc = []
best_auc = []

# training and testing
start_time = time.perf_counter()
for epoch in range(EPOCH):
    for step, (train_x, train_y) in enumerate(train_loader):
        
        train_x= train_x.transpose(1,2)
        output = lstm(train_x)  # LSTM output of training data
        loss = loss_func(output, train_y.long())  # cross entropy loss
        optimizer.zero_grad()  # clear gradients for this training step
        loss.backward()  # backpropagation, compute gradients
        optimizer.step()  # apply gradients

        test_fea_norm1= test_fea_norm1.transpose(1,2)
        
        if epoch % 10 == 0 and step==2:
            
            test_output = lstm(test_fea_norm1)  # LSTM output of test data
            test_loss = loss_func(test_output, test_label.long())

            test_y_score = one_hot(test_label.data.cpu().numpy())  # .cpu() can be removed if your device is cpu.
            pred_score = F.softmax(test_output, dim=1).data.cpu().numpy()  # normalize the output
            auc_score = roc_auc_score(test_y_score, pred_score)

            pred_y = torch.max(test_output, 1)[1].data.cpu().numpy()
            pred_train = torch.max(output, 1)[1].data.cpu().numpy()

            test_acc = accuracy_score(test_label.data.cpu().numpy(), pred_y)
            train_acc = accuracy_score(train_y.data.cpu().numpy(), pred_train)


            print('Epoch: ', epoch, '|train loss: %.4f' % loss.item(),
                  ' train ACC: %.4f' % train_acc, '| test loss: %.4f' % test_loss.item(),
                  'test ACC: %.4f' % test_acc, '| AUC: %.4f' % auc_score)
            best_acc.append(test_acc)
            best_auc.append(auc_score)

current_time = time.perf_counter()
running_time = current_time - start_time
print(classification_report(test_label.data.cpu().numpy(), pred_y))
print('BEST TEST ACC: {}, AUC: {}'.format(max(best_acc), max(best_auc)))
print("Total Running Time: {} seconds".format(round(running_time, 2)))

Epoch:  0 |train loss: 2.3081  train ACC: 0.0721 | test loss: 2.3136 test ACC: 0.0541 | AUC: 0.4969
Epoch:  10 |train loss: 2.3071  train ACC: 0.1081 | test loss: 2.3160 test ACC: 0.0811 | AUC: 0.4637
Epoch:  20 |train loss: 2.3080  train ACC: 0.0991 | test loss: 2.3129 test ACC: 0.0901 | AUC: 0.5444
Epoch:  30 |train loss: 2.3069  train ACC: 0.1261 | test loss: 2.3125 test ACC: 0.0541 | AUC: 0.5200
Epoch:  40 |train loss: 2.3080  train ACC: 0.0811 | test loss: 2.3150 test ACC: 0.0721 | AUC: 0.4858
Epoch:  50 |train loss: 2.3051  train ACC: 0.0991 | test loss: 2.3130 test ACC: 0.0901 | AUC: 0.5081
Epoch:  60 |train loss: 2.3061  train ACC: 0.1351 | test loss: 2.3130 test ACC: 0.0631 | AUC: 0.5011
Epoch:  70 |train loss: 2.3038  train ACC: 0.0991 | test loss: 2.3132 test ACC: 0.0721 | AUC: 0.5237
Epoch:  80 |train loss: 2.3062  train ACC: 0.0991 | test loss: 2.3127 test ACC: 0.0901 | AUC: 0.4952
Epoch:  90 |train loss: 2.3075  train ACC: 0.0901 | test loss: 2.3125 test ACC: 0.1171 | AUC

KeyboardInterrupt: 